<a href="https://colab.research.google.com/github/DevDeividMoura/GemHelper/blob/main/GemHelper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GemHelper - O Assistente Ideal para Otimizar a UX da Sua Empresa

## Autoria
- **Autor:** Deivid Alexsander Carvalho Moura
- **Evento:** Imersão AI Alura + Google
- **Data:** 11/05/2024
- **Contato:** www.linkedin.com/in/devdeividmoura



## 1 - Introdução


### 1.1 - Contexto

Atualmente, muitas empresas utilizam sistemas automatizados de atendimento ao cliente, como bots, para gerenciar as primeiras interações com os usuários. Essa abordagem ajuda a reduzir a carga de trabalho sobre atendentes humanos e acelerar a resolução de questões simples. No entanto, nem todas as solicitações podem ser completamente resolvidas por bots, levando à necessidade de transferir a interação para um atendente humano. Esta transferência frequentemente exige que o atendente releia toda a interação anterior do usuário com o bot ou com outros atendentes para entender o contexto da solicitação e identificar o cliente, um processo que consome tempo valioso e reduz a eficiência do atendimento.

### 1.2 - Problema

A revisão do histórico de chat pelo atendente não apenas consome tempo, mas também pode levar à repetição de perguntas já respondidas, frustrando o cliente e tornando a experiência menos produtiva e agradável. A necessidade de otimizar esse processo é clara, tanto para melhorar a eficiência dos atendentes quanto para elevar a satisfação do cliente.

### 1.3 - Solução Proposta
Este projeto visa desenvolver um modelo de inteligência artificial que possa automaticamente gerar um resumo conciso do histórico de chat entre o cliente e o sistema de atendimento (bots e atendentes anteriores). O resumo incluirá pontos-chave como a identificação do cliente, a natureza da solicitação, o status atual do atendimento e os próximos passos recomendados. Com isso, pretendemos reduzir significativamente o tempo necessário para que novos atendentes se familiarizem com o contexto de cada caso, acelerando o processo de atendimento e melhorando a experiência do usuário.

### 1.4 - Metodologia
Para alcançar este objetivo, utilizaremos uma combinação de técnicas de "Chain of Thoughts" (Cadeia de Pensamentos) e aprendizado por few-shot. Isso envolve treinar o modelo em exemplos de interações de chat junto com resumos estruturados que destacam informações essenciais. O modelo, portanto, aprenderá a identificar e extrair as informações mais relevantes dos diálogos, formatando-as em um resumo útil e prático.

## 2 - Preparando o modelo Gemini
### 2.1 - Importação e Configurações do Modelo

Neste notebook, o modelo Gemini 1.5 pro é usado atraves da API do Google Cloud e manipulado com o SDK oficial disponibilizado pelo google.



In [8]:
import google.generativeai as genai
from google.colab import userdata

API_KEY = userdata.get("API_KEY")
genai.configure(api_key=API_KEY)

MODEL_NAME = "gemini-1.5-pro-latest"

generation_config = {
  "temperature": 1,
  "top_p": 0.98,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]


model = genai.GenerativeModel(model_name=MODEL_NAME,
                              generation_config=generation_config,
                              safety_settings=safety_settings)

### 2.2 - Executando o modelo Gemini diretamente
Os modelos Gemma podem gerar sentenças da seguinte forma,

In [15]:
response = model.generate_content(
    "Olá, sendo breve diga-me o que você é capaz de  fazer?")
print(response.text)


Sou um modelo de linguagem grande, treinado pelo Google.

**Posso:**

* Gerar texto criativo e informativo
* Responder a perguntas e fornecer informações
* Traduzir idiomas
* Resumir texto
* Escrever diferentes tipos de conteúdo (e-mails, artigos, poemas, etc.)
* Ajudar com tarefas de codificação
* E muito mais!

**Não posso:**

* Pensar ou sentir emoções
* Ter experiências do mundo real
* Dar conselhos médicos ou financeiros
* Substituir a interação humana

Em resumo, sou uma ferramenta poderosa para comunicação e criação, mas não sou um ser humano. 😄



## 3 - Dados de Exemplo
### 3.1 - Importação dos Dados

Antes de treinar o modelo para gerar resumos úteis dos chats, precisamos carregar e preparar o conjunto de dados que será utilizado. Nesta seção, vamos importar o conjunto de dados de exemplo, que contém interações de chat entre clientes e atendentes, bem como os resumos esperados.


In [19]:
import pandas as pd

# Carregar os dados de um arquivo Excel
df_exemplos = pd.read_excel('dataset_exemplos.xlsx')

Quantos exemplos existem?

In [18]:
print(df_exemplos.shape[0])

7


OK, 7 exemplos para iniciarmos

In [21]:
display(df_exemplos)

,input,output
0,Isabela Gonçalves 09/05/2024 - Oi\nAssistente ...,**Resumo do Atendimento: Isabela Gonçalves - 0...
1,Paulo Martins 08/05/2024 - Boa noite \nAssist...,**Resumo do Atendimento: Paulo Martins - 08/05...
2,Carlos Silva 10/05/2024 - Boa noite\nAssistent...,## Resumo do Atendimento: Carlos Silva - 10/05...
3,Mariana Souza 10/05/2024 - Ola boa tarde\nAssi...,## Resumo do Atendimento: Mariana Souza - 10/0...
4,"VIRTUAL Suporte 3 10/05/2024 - Boa tarde, tudo...",## Resumo do Atendimento: Gabriel Santos/Lucia...
5,Carlos Almeida 10/05/2024 - Bom dia\nAssistent...,## Resumo do Atendimento: Carlos Almeida - 10/...
6,Larissa Gomes 08/05/2024 - Oi\nAssistente Virt...,## Resumo do Atendimento: Larissa Gomes - 08/0...


## 4 - Criando o Summarizer com Técnicas Avançadas de Engenharia de Prompt
### 4.1 - Introdução às Técnicas
Nesta seção, utilizaremos duas técnicas avançadas de engenharia de prompt apresentada na imerção para aprimorar a capacidade do modelo de gerar resumos eficazes a partir de interações de chat.

- **Chain-of-Thought**: Esta técnica envolve a geração de uma série de passos de raciocínio intermediários que ajudam os modelos de linguagem a realizar tarefas complexas de raciocínio. Essencialmente, ela guia o modelo para pensar passo a passo, melhorando a qualidade das respostas geradas.

- **Few-Shot Prompting**: Esta técnica explora a capacidade de modelos de linguagem, como o Gemini, para realizar tarefas com apenas alguns exemplos de treinamento. Ao invés de requerer grandes datasets para treinamento, apenas alguns exemplos são suficientes para instruir o modelo a realizar uma nova tarefa.

### 4.2 - Configuração das Instruções de Chain-of-Thought

O objetivo principal do modelo é otimizar o fluxo de atendimentos, gerando resumos concisos e diretamente relevantes que capturem as informações essenciais de cada interação. As instruções detalhadas abaixo guiarão o modelo no processo de geração de resumos, utilizando a técnica de Chain-of-Thought para estruturar o raciocínio de maneira lógica e sequencial.



In [22]:
instructions = """
Como assistente de suporte, seu principal objetivo é otimizar o fluxo de
atendimentos gerando resumos concisos e diretamente relevantes.
Estes resumos devem incluir apenas as informações essenciais: solicitações do
cliente, dados coletados, status atual do atendimento e próximos passos de
maneira prática. Certifique-se de que cada interação esteja claramente
documentada para evitar repetições desnecessárias e facilitar a transição entre
atendentes. Baseie suas ações no contexto atual para agilizar cada passo do
processo. Mantenha o resumo breve, direto e totalmente alinhado às necessidades
expressas pelo cliente, garantindo uma comunicação eficaz e eficiente no suporte
ao cliente. Primeiro, apresentarei a estrutura do resumo e exemplos de chats de
atendimentos com seus respectivos resumos. Analise esses exemplos para
compreender os pontos relevantes e a estrutura desejada antes de resumir o chat
desafio proposto.


ESTRUTURA DO RESULMO:

Resumo do Atendimento: <NOME DO CONTATO> - <DATA-ATENDIMETNO>

Problema Reportado:<DESCRIÇÃO SIMPLES DO COM MINIMO DE PALAVRAS POSSIVEIS>.
Dados Coletados:
     CPF: <CPF-CAPTURADO>
     Cidade escolhida: <CIDADE>
Status Atual: <STATUS ATUAL DO ATENDIMENTO>.
Observações: <SUGESTÃO DE MELHORIA NO BOT SE HOUVER>.
Próximos Passos:
     <PROXIMOS PASSOS A SEREM FEITOS PELO ATENDENTE>
"""

### 4.3 - Implementação de Few-Shot Prompting
Utilizaremos a técnica de Few-Shot Prompting para treinar o modelo a partir de poucos exemplos, demonstrando como gerar resumos a partir de interações reais. Cada exemplo no conjunto de dados servirá como um prompt para ensinar o modelo a sintetizar informações cruciais em um formato estruturado e útil.


In [29]:
prompt_parts_base = [instructions]  # Incluir as instruções detalhadas

for index, row in df_exemplos.iterrows():
    # Cada entrada no dataframe é transformada em um prompt de treinamento
    prompt_parts_base.append(
        f"input: EXEMPLO DE ATENDIMENTO {index + 1}:\n{row['input']}")
    prompt_parts_base.append(
        f"output: RESUMO DO EXEMPLO DE ATENDIMENTO {index + 1}:\n{row['output']}")

In [30]:
for part in prompt_parts_base[:3]:
  print(part)


Como assistente de suporte, seu principal objetivo é otimizar o fluxo de 
atendimentos gerando resumos concisos e diretamente relevantes. 
Estes resumos devem incluir apenas as informações essenciais: solicitações do 
cliente, dados coletados, status atual do atendimento e próximos passos de 
maneira prática. Certifique-se de que cada interação esteja claramente 
documentada para evitar repetições desnecessárias e facilitar a transição entre 
atendentes. Baseie suas ações no contexto atual para agilizar cada passo do 
processo. Mantenha o resumo breve, direto e totalmente alinhado às necessidades 
expressas pelo cliente, garantindo uma comunicação eficaz e eficiente no suporte 
ao cliente. Primeiro vou lhe mostrar a estrutura do resumo e exemplos de chats 
de atendimentos junto com exemplos de resumos gerados apartir de cada chat, 
em seguida analise os exemplos e entenda os pontos relevantes e a estrutura 
desejada, e porfim resuma o chat proposto.

ESTRUTURA DO RESULMO:

Resumo do A

As instruções e exemplos configurados aqui são fundamentais para direcionar o modelo na aprendizagem de como transformar diálogos complexos em resumos claros e concisos, fundamentais para otimizar a experiência de atendimento ao cliente.


Essa formatação melhora a legibilidade e a organização das instruções, além de deixar claros os objetivos e a metodologia que seu modelo deve seguir para gerar resumos eficazes.


##5 - Sumarização de Chats
###5.1 - Configuração do Prompt de Sumarização
Vamos preparar um prompt de sumarização que integra tanto o input (a interação de chat) quanto o processo de raciocínio (Chain-of-Thought), seguido pelo pedido de geração do resumo. Essa etapa utiliza os prompts de treinamento que preparamos anteriormente para guiar o modelo na geração de resumos úteis.

In [7]:
chat_challenge_1 = """
Laura Silva 10/05/2024 - Boa noite
Assistente Virtual 10/05/2024 - Boa noite, seja bem vindo ao atendimento da VIRTUAL NET.
Assistente Virtual 10/05/2024 - Em que posso te ajudar?
Laura Silva 10/05/2024 - Quero efetuar pagamento
Assistente Virtual 10/05/2024 - Informe por gentileza o CPF ou CNPJ do titular do contrato.
Laura Silva 10/05/2024 - 123.456.789-01
Assistente Virtual 10/05/2024 - Por favor, selecione um boleto abaixo: 1 - Vencimento: 10/04/2024 2 - Vencimento: 10/05/2024 3 - Vencimento: 10/06/2024
Laura Silva 10/05/2024 - 1
Assistente Virtual 10/05/2024 - Segue abaixo as informações do seu boleto: [Link Fictício]
Assistente Virtual 10/05/2024 - Eu posso te ajudar em algo mais 😊? 1 - Sim 2 - Não 3 - Outro boleto 4 - Falar com um atendente
Laura Silva 10/05/2024 - Falar com um atendente
Assistente Virtual 10/05/2024 - Selecione a cidade que deseja atendimento. 1 - Cidade A 2 - Cidade B 3 - Cidade C 4 - Cidade D 5 - Cidade E 6 - Cidade F 7 - Cidade G
Laura Silva 10/05/2024 - Cidade A
Assistente Virtual 10/05/2024 - Será necessário transferir o seu atendimento para nossa equipe especializada que lhe dará todo o suporte. Por favor, aguarde um momento...
Plantão 1 10/05/2024 - Olá Laura, Boa noite! ☺️ Aqui quem fala é o Carlos, da equipe de atendimento, tudo bom?
Plantão 1 10/05/2024 - Laura, como eu posso te ajudar hoje?
Laura Silva 10/05/2024 - Boa noite tudo bem
Laura Silva 10/05/2024 - Quero o pix de vocês para efetuar o pagamento da parcela
Plantão 1 10/05/2024 - Entendi Laura, terei que encaminhar seu atendimento para o financeiro providenciar a chave pix, tá bem?
Assistente Virtual 10/05/2024 - Transferido para VIRTUAL financeiro 1
"""

prompt_parts_chat_1 = prompt_parts_base + [
  f"input: CHAT DESAFIO: {chat_challenge_1}",
  "output: "
]

> ## Resumo do Atendimento: Laura Silva - 10/05/2024
> 
> **Solicitação Inicial:** Efetuar pagamento.
> 
> **Dados Coletados:**
> * CPF: 123.456.789-01
> * Boleto visualizado: Vencimento 10/04/2024
> * Cidade Escolhida: Cidade A
> 
> **Status Atual:** 
> * Cliente visualizou boleto de vencimento 10/04/2024.
> * Cliente solicitou contato com atendente.
> * Transferido para equipe especializada (Plantão 1 - Carlos).
> * Cliente solicitou chave PIX para pagamento.
> * Transferido para equipe financeira (VIRTUAL financeiro 1).
> 
> **Observações:** Nenhuma.
> 
> **Próximos Passos:**
> * Equipe financeira deve gerar e fornecer a chave PIX para a cliente. 


### 5.2 - Execução do Modelo para Geração de Sumário
Agora, vamos utilizar o modelo configurado para gerar o sumário do desafio proposto, baseando-se no prompt completo que inclui o raciocínio e o exemplo do chat desafio.

In [31]:
response = model.generate_content(prompt_parts_chat_1)
print(response.text)

## Resumo do Atendimento: Laura Silva - 10/05/2024

**Solicitação Inicial:** Efetuar pagamento.

**Dados Coletados:**
* CPF: 123.456.789-01
* Boleto visualizado: Vencimento 10/04/2024
* Cidade Escolhida: Cidade A

**Status Atual:** 
* Cliente solicitou contato com atendente após visualizar boleto.
* Transferido para equipe especializada (Plantão 1 - Carlos).
* Cliente solicitou chave PIX para pagamento.
* Transferido para o setor financeiro (VIRTUAL financeiro 1).

**Observações:** Nenhuma.

**Próximos Passos:**
* Aguardar atendimento do setor financeiro (VIRTUAL financeiro 1) para fornecer a chave PIX e finalizar o pagamento. 



### 5.3 - Visualização de Resumos com Markdown
Para melhorar a visualização e a leitura dos resumos gerados pelo nosso modelo, utilizamos uma função especial em Python que converte texto simples em formato Markdown, aplicando indentação e substituindo caracteres específicos para melhorar a formatação visual no Jupyter Notebook. Essa abordagem é especialmente útil para destacar informações importantes de maneira clara e organizada, tornando a análise dos resultados mais acessível e compreensível.


In [32]:

import pandas as pd
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


display(to_markdown(response.text))

> ## Resumo do Atendimento: Laura Silva - 10/05/2024
> 
> **Solicitação Inicial:** Efetuar pagamento.
> 
> **Dados Coletados:**
> * CPF: 123.456.789-01
> * Boleto visualizado: Vencimento 10/04/2024
> * Cidade Escolhida: Cidade A
> 
> **Status Atual:** 
> * Cliente solicitou contato com atendente após visualizar boleto.
> * Transferido para equipe especializada (Plantão 1 - Carlos).
> * Cliente solicitou chave PIX para pagamento.
> * Transferido para o setor financeiro (VIRTUAL financeiro 1).
> 
> **Observações:** Nenhuma.
> 
> **Próximos Passos:**
> * Aguardar atendimento do setor financeiro (VIRTUAL financeiro 1) para fornecer a chave PIX e finalizar o pagamento. 


In [36]:
chat_challenge_2 = """
Rogerio Ulka 09/05/2024 - Olá
Assistente Virtual 09/05/2024 - Boa tarde, seja bem vindo ao atendimento da VIRTUAL NET.
Assistente Virtual 09/05/2024 - Em que posso te ajudar?
Rogerio Ulka 09/05/2024 - Gostaria de saber valores dos planos
Assistente Virtual 09/05/2024 - Selecione a cidade que deseja atendimento. 1 - Cidade A 2 - Cidade B 3 - Cidade C 4 - Cidade D 5 - Cidade E 6 - Cidade F 7 - Cidade G
Rogerio Ulka 09/05/2024 - E taxa de instalação
Assistente Virtual 09/05/2024 - Digite uma opção válida por gentileza.
Assistente Virtual 09/05/2024 - Selecione a cidade que deseja atendimento. 1 - Cidade A 2 - Cidade B 3 - Cidade C 4 - Cidade D 5 - Cidade E 6 - Cidade F 7 - Cidade G
Rogerio Ulka 09/05/2024 - Cidade D
Assistente Virtual 09/05/2024 - Será necessário transferir o seu atendimento para nossa equipe especializada que lhe dará toda a atenção merecida. Por favor, aguarde um momento...
Comercial VIRTUAL 1 09/05/2024 - Boa tarde, em que posso ajudar?
Rogerio Ulka 09/05/2024 - Gostaria de saber quais os planos vocês oferecem
Rogerio Ulka 09/05/2024 - E se tem taxa de instalação
Comercial VIRTUAL 1 09/05/2024 - Qual seria seu endereço?
Rogerio Ulka 09/05/2024 - Estrada do Sol km 84
"""

prompt_parts_chat_2 = prompt_parts_base + [
  f"input: CHAT DESAFIO: {chat_challenge_2}",
  "output: "
]

response = model.generate_content(prompt_parts_chat_2)
display(to_markdown(response.text))

> ## Resumo do Atendimento: Rogerio Ulka - 09/05/2024
> 
> **Solicitação:** Consultar valores dos planos e taxa de instalação.
> 
> **Dados Coletados:**
> * Cidade Escolhida: Cidade D
> * Endereço: Estrada do Sol km 84
> 
> **Status Atual:** Transferido para equipe especializada (Comercial VIRTUAL 1).
> 
> **Observações:** 
> * Bot solicitou a cidade antes da solicitação completa do cliente, o que pode ser otimizado.
> 
> **Próximos Passos:**
> * Consultar a disponibilidade de planos e valores para o endereço informado.
> * Informar ao cliente os planos disponíveis e seus respectivos valores, incluindo a taxa de instalação. 


## 6 - Conclusão

Este projeto demonstrou a capacidade do modelo Gemini, combinado com técnicas avançadas de engenharia de prompt como Chain-of-Thought e Few-Shot Prompting, para otimizar o atendimento ao cliente através da sumarização de interações de chat. Ao implementar essas técnicas, conseguimos reduzir significativamente o tempo necessário para que atendentes compreendam o contexto de cada atendimento, melhorando tanto a eficiência do processo quanto a experiência do usuário final.

### 6.1 - Principais Aprendizados
- A aplicação de técnicas de IA avançadas pode transformar processos de atendimento ao cliente, tornando-os mais rápidos e eficazes.
- A preparação e estruturação de dados são fundamentais para treinar modelos de linguagem de forma eficiente.

### 6.2 - Próximos Passos
- Realizar mais testes com diferentes configurações de modelos para explorar formas de aumentar ainda mais a precisão dos resumos.
- Expandir o dataset de treinamento com mais exemplos para melhorar a generalização do modelo.
- Investigar a integração deste sistema com plataformas de atendimento ao cliente em tempo real para avaliar a aplicabilidade prática em ambientes empresariais.
- Desenvolver extensões de navegador com um chat que possam acessar automaticamente as conversas em contexto nas plataformas de atendimento. Essas extensões poderiam automatizar o envio de conversas ao modelo e o recebimento de resumos, proporcionando aos atendentes uma visão imediata e precisa do contexto do cliente assim que abrem um atendimento, potencializando ainda mais a eficiência do processo.


Ao continuar desenvolvendo e aprimorando ferramentas como o GemHelper, podemos esperar um impacto cada vez maior na forma como as empresas interagem com seus clientes, levando a uma satisfação significativamente melhorada e a operações mais eficientes.
